In [1]:
import pandas as pd
import io
import glob

# 1. AYARLAR: Süreyi saniyeye çeviren fonksiyon
def hms_to_sec(t):
    t = str(t).strip()
    if not t or t == 'nan': return 0
    days = 0
    if 'g' in t: 
        parts = t.split('g')
        try:
            days = int(parts[0].strip())
            t = parts[1].strip()
        except: pass
    parts = t.split(':')
    try:
        if len(parts) == 3:
            return days*86400 + int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])
        elif len(parts) == 2:
            return days*86400 + int(parts[0])*60 + int(parts[1])
    except: return 0
    return 0

# 2. DOSYALARI BUL VE TEMİZLEYEREK BİRLEŞTİR
dosyalar = sorted(glob.glob("all_merged_data*.csv"))
print(f"Tespit edilen parçalar: {dosyalar}")

sutun_isimleri = ['#', 'vehicle_id', 'Enlem', 'Boylam', 'Duraklama Süresi', 'Rölanti Süresi', 
                  'Yükseklik', 'Durum', 'Açıklama', 'Tarih', 'Saat', 'Gun', 
                  'Hız(km/sa)', 'Mesafe(km)', 'Mesafe Sayacı(km)', 'Adres', 'Mahalle', 'Kaynak']

parca_listesi = []

for dosya in dosyalar:
    print(f"{dosya} işleniyor...")
    temiz_satirlar = []
    with open(dosya, 'r', encoding='utf-8-sig') as f:
        for line in f:
            line = line.strip()
            if line.startswith('"') and line.endswith('"'):
                line = line[1:-1]
            line = line.replace('""', '"')
            
            if line.startswith('#') or not line or 'vehicle_id' in line:
                continue
            temiz_satirlar.append(line)
    
    df_parca = pd.read_csv(io.StringIO("\n".join(temiz_satirlar)), header=None, names=sutun_isimleri)
    parca_listesi.append(df_parca)

df = pd.concat(parca_listesi, ignore_index=True)

# 3. KONTEYNIR NOKTALARINI FİLTRELE
df['Saniye'] = df['Duraklama Süresi'].apply(hms_to_sec)

# Kriter: 30 saniye ile 10 dakika arası duraklamalar
konteyner_filtresi = df[(df['Saniye'] >= 30) & (df['Saniye'] <= 600)].copy()

# Tekilleştirme için geçici yuvarlama kolonları
konteyner_filtresi['Lat_rnd'] = konteyner_filtresi['Enlem'].round(4)
konteyner_filtresi['Lon_rnd'] = konteyner_filtresi['Boylam'].round(4)

# Mükerrerleri temizle (6621 benzersiz nokta kalacak)
son_liste = konteyner_filtresi.drop_duplicates(subset=['Lat_rnd', 'Lon_rnd']).copy()

# 4. TEMİZLİK VE KAYDETME
# Analiz için kullandığımız geçici kolonları siliyoruz ki dosya orijinal formatta kalsın
son_liste = son_liste.drop(columns=['Saniye', 'Lat_rnd', 'Lon_rnd'])

cikti_dosyasi = 'tum_konteyner_lokasyonlari_detayli.csv'
# Seçim yapmadan TÜM kolonları kaydediyoruz
son_liste.to_csv(cikti_dosyasi, index=False, encoding='utf-8-sig')

print(f"İşlem tamamlandı! {len(son_liste)} adet nokta ve tüm sütunlar '{cikti_dosyasi}' dosyasına kaydedildi.")

Tespit edilen parçalar: ['all_merged_data.csv']
all_merged_data.csv işleniyor...
İşlem tamamlandı! 28072 adet nokta ve tüm sütunlar 'tum_konteyner_lokasyonlari_detayli.csv' dosyasına kaydedildi.


In [2]:
import pandas as pd

# 1. Dosyaları yükle
# tum_konteyner_lokasyonlari_detayli.csv dosyanızın klasörde olduğundan emin olun
konteyner_df = pd.read_csv('tum_konteyner_lokasyonlari_detayli.csv')
fleet_df = pd.read_csv('fleet.csv')

# 2. Fleet dosyasından sadece eşleştirme için gereken sütunları alalım
# vehicle_id üzerinden giderek yanına vehicle_type sütununu ekleyeceğiz
fleet_mapping = fleet_df[['vehicle_id', 'vehicle_type']].drop_duplicates()

# 3. Sol Birleştirme (Left Merge) işlemi
# Bu işlem ana dosyanızdaki tüm satırları korur ve her vehicle_id'nin yanına karşılık gelen tipi yazar
son_df = pd.merge(konteyner_df, fleet_mapping, on='vehicle_id', how='left')

# 4. Tüm kolonların olduğu yeni dosyayı kaydet
son_df.to_csv('konteyner_lokasyonlari_tipli.csv', index=False, encoding='utf-8-sig')

print(f"İşlem başarıyla tamamlandı!")
print(f"Yeni dosya 'konteyner_lokasyonlari_tipli.csv' adıyla kaydedildi.")
print(f"Toplam sütun sayısı: {len(son_df.columns)}")

İşlem başarıyla tamamlandı!
Yeni dosya 'konteyner_lokasyonlari_tipli.csv' adıyla kaydedildi.
Toplam sütun sayısı: 19
